<a href="https://colab.research.google.com/github/vishal-gupta025/Twitter-Sentiment-Analysis-in-ML/blob/main/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.19GB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
cols = ['target','ids','date','flag','user','text']
df.columns = cols

In [ ]:
df.shape

(1599999, 6)

In [ ]:
df.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.drop(columns=['ids','date','flag','user'],inplace=True)

In [ ]:
df

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew
...,...,...
1599994,4,Just woke up. Having no school is the best fee...
1599995,4,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,Happy 38th Birthday to my boo of alll time!!! ...


from matplotlib import pyplot as plt
df['target'].plot(kind='hist', bins=20, title='target')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['target']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'target'}, axis=1)
              .sort_values('target', ascending=True))
  xs = counted['target']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('target', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('target')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
df['target'].plot(kind='line', figsize=(8, 4), title='target')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
df.target = df.target.replace(4,1)

In [ ]:
df.target.value_counts()

,count
target,
1,800000
0,799999


In [ ]:
ps = PorterStemmer()
def clean_text(content):
  text = re.sub('[^a-zA-Z]',' ',content)
  text = text.lower()
  text = text.split()
  text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
  text = ' '.join(text)
  return text

In [ ]:
df['stemmed_text'] = df['text'].apply(clean_text)

In [ ]:
print(df.stemmed_text)

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_text, Length: 1599999, dtype: object


In [ ]:
df.shape

(1599999, 3)

In [ ]:
x = df.stemmed_text.values
y = df.target.values

In [ ]:
print(x)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ... 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [ ]:
print(x.shape,X_train.shape,X_test.shape)

(1599999,) (1279999,) (320000,)


In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453002 stored elements and shape (1279999, 461471)>
  Coords	Values
  (0, 436655)	0.27247657460069113
  (0, 354566)	0.358082129508793
  (0, 185125)	0.5296934860731303
  (0, 109298)	0.37470892870977707
  (0, 234920)	0.4196620311217523
  (0, 443035)	0.44770025018080273
  (1, 166826)	0.3977368489124923
  (1, 19762)	0.25286013616648123
  (1, 119689)	0.4309123985667578
  (1, 152015)	0.49196931362728563
  (1, 319721)	0.3128279791558143
  (1, 86730)	0.33015139495167095
  (1, 172343)	0.3785321144028949
  (2, 326288)	0.5853283798404598
  (2, 220710)	0.2626149875630722
  (2, 279017)	0.28499749936226054
  (2, 445635)	0.2356370052189603
  (2, 412554)	0.28961919890983545
  (2, 124728)	0.3317593891539697
  (2, 117914)	0.39861155079647814
  (2, 146003)	0.22130486737209085
  (2, 93826)	0.22330494345030885
  (3, 93826)	0.46202690964703447
  (3, 331805)	0.4656145237310222
  (3, 46285)	0.6887696820070169
  :	:
  (1279996, 13878)	0.43588823915

In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289342 stored elements and shape (320000, 461471)>
  Coords	Values
  (0, 63768)	0.26422682589785035
  (0, 75261)	0.26552138110472423
  (0, 127038)	0.2730995033465573
  (0, 219133)	0.3295455088887744
  (0, 320624)	0.5238287544743887
  (0, 355868)	0.22363572756759686
  (0, 371332)	0.3972681545090477
  (0, 375613)	0.24658637821166524
  (0, 382367)	0.2541115193458825
  (0, 414033)	0.2624690535388268
  (1, 6452)	0.3061987253241848
  (1, 15098)	0.21044041574811612
  (1, 145408)	0.5774478727480551
  (1, 217431)	0.40260807391422926
  (1, 256686)	0.2861349119561766
  (1, 348128)	0.4737202797635419
  (1, 366259)	0.24522188688615762
  (2, 79341)	0.34512546063620464
  (2, 88422)	0.33529803711277506
  (2, 107866)	0.36857211925047617
  (2, 124443)	0.2729825984376057
  (2, 258609)	0.3072614985829275
  (2, 375890)	0.3020734075258265
  (2, 407333)	0.27060796399382786
  (2, 418804)	0.33257344872506
  :	:
  (319995, 193560)	0.373116119782695


In [ ]:
lg = LogisticRegression(max_iter=1000)

In [ ]:
lg.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
y_pred = lg.predict(X_test)

In [ ]:
accuracy_score(Y_test,y_pred)

0.7771625